In [1]:
import pandas as pd
import os
import email
import time
import csv

#unique delim to separate columns without messing up content
DELIMETER = chr(255)

In [24]:
root_path = '/home/peter/Desktop/maildir'
saved_file = '/home/peter/Desktop/processed enron emails.csv'
testing_batch = False

In [25]:
#list for holding file paths
list_of_files = []

#safety check to see if the file has been created so I don't overwrite a finished file
already_processed = os.path.isfile(saved_file) 

#get the path for all the email files
for path, subdirs, files in os.walk(root_path):
    for name in files:
        list_of_files.append(os.path.join(path, name))

email_count = len(list_of_files)
#caluculate 1% of the number of files, used for progress report printing.
onepcofemail = (email_count//100)
        
#if no file already exists start the writing process
if not already_processed:
    
    #create dataframe to store data to be written
    emails = pd.DataFrame()
    print('Starting. {}'.format(time.strftime("%I:%M:%S")))
    
    #counter for testing batches
    x = 0
    
    #iterrate through all the files found
    for i, myfile in enumerate(list_of_files):
        
        #if running a testing session to break before reading all file to inspect the saved csv
        if x > 6000 and testing_batch: break
        
        #read individual email file
        with open(myfile, 'r', encoding='utf-8', errors='replace') as filepath:
            #read in the email
            message=email.message_from_string(filepath.read())

            #save the important components
            emails.loc[i,'Message-ID']=message['Message-ID']
            emails.loc[i,'from']=message['from']
            emails.loc[i,'subject']=message['subject']
            emails.loc[i,'to']=message['to']
            emails.loc[i,'cc']=message['cc']
            emails.loc[i,'bcc']=message['Bcc']
            emails.loc[i,'date']=message['date']
            emails.loc[i,'file']=message['X-FileName']
            
            #check that the body is one or multiple sections and save it
            if message.is_multipart():
                for payload in message.get_payload():
                    print(payload.get_payload())
                    string = ''
                    string = string + payload.get_payload()
                    emails.loc[i,'body'] = string
            else:
                emails.loc[i,'body'] =  message.get_payload()
            
        #append entry to the csv file
        with open(saved_file, 'a') as f:
            #if this is the first entry use the headers, if not, dont.True
            if i == 0: emails.to_csv(f, header=emails.columns, index = False, sep=DELIMETER)
            else: emails.to_csv(f, header=None, index = False, sep=DELIMETER)

        #clear the dataframe to save memory
        emails = pd.DataFrame()        

        #print statement updating progress report
        if i % onepcofemail == 0: 
            print('{}% finished. {}'.format(i//onepcofemail, time.strftime("%I:%M:%S")), end="\r")

        #itterate counter for testing batches
        x = x+1
            
                        
    #process complete
    print('Done! {}'.format(time.strftime("%I:%M:%S")))

#prints if the file already exists and there is no need to process the individual emails into a csv
else: print('****************File Previously Processed, Delete File to Process Again********************')

Starting. 04:09:52
Done! 06:59:56 06:59:56


In [22]:
# print('Loading File. {}'.format(time.strftime("%I:%M:%S")))
# df = pd.read_csv(saved_file, sep=DELIMETER, engine='python')
# print('File Loaded! {}'.format(time.strftime("%I:%M:%S")))